In [ ]:
import sys
# path-ul catre dataset-ul cu cod
sys.path.insert(1, '/kaggle/input/pycode')

In [ ]:
# Fisierul generic_dataset.py modificat
# import glob
# import ntpath
# import random
# from PIL import Image

# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms as T


# CELEBA_FORMAT_DATASET = 1
# RAFD_FORMAT_DATASET = 2


# class GenericDataset(Dataset):
#     """Dataset description class"""
#     def __init__(self, path, format, selected_attrs, mode, **kwargs):
#         self.path = path
#         self.format = format
#         self.selected_attrs = selected_attrs
#         self.mode = mode
#         self.transform = kwargs.get("transform", get_default_transforms(mode))

#         self.attr2idx = {}
#         self.idx2attr = {}
#         self.train_dataset = []
#         self.test_dataset = []

#         if self.format == CELEBA_FORMAT_DATASET:
#             self.preprocess_celeba()
#         else:
#             self.preprocess_rafd()

#     def preprocess_celeba(self):
#         lines = [line.strip() for line in open("/kaggle/input/pycode/list_attr.txt", "r")]
#         cnt_samples, all_attr_names = int(lines[0]), lines[1].split()
#         assert set(self.selected_attrs).issubset(set(all_attr_names)), "Invalid selected attrs"
#         for i, attr_name in enumerate(all_attr_names):
#             self.attr2idx[attr_name] = i
#             self.idx2attr[i] = attr_name

#         lines = lines[2:]
#         random.seed(1234)
#         random.shuffle(lines)
#         for i, line in enumerate(lines):
#             split = line.split()
#             filename = split[0]
#             values = split[1:]

#             label = []
#             for attr_name in self.selected_attrs:
#                 idx = self.attr2idx[attr_name]
#                 label.append(values[idx] == '1')

#             if (i+1) <= max(1, cnt_samples // 50):
#                 self.test_dataset.append([self.path + "/img_align_celeba/img_align_celeba/" + filename, label])
#             else:
#                 self.train_dataset.append([self.path + "/img_align_celeba/img_align_celeba/" + filename, label])

#         print(f'Finished preprocessing CelebA-format dataset at {self.path}: {len(self.train_dataset)} train images, {len(self.test_dataset)} test images')

#     def preprocess_rafd(self):
#         categs = set(ntpath.basename(categ_folder) for categ_folder in glob.glob(self.path + "/train/*"))
#         categs_test = set(ntpath.basename(categ_folder) for categ_folder in glob.glob(self.path + "/test/*"))
#         assert categs == categs_test, "Train and test do not have the same categories"
#         assert set(self.selected_attrs).issubset(categs), "Invalid selected attrs"
#         for i, attr_name in enumerate(categs):
#             self.attr2idx[attr_name] = i
#             self.idx2attr[i] = attr_name

#         for subfolder in ["train", "test"]:
#             dataset = self.train_dataset if subfolder == "train" else self.test_dataset
#             for categ in categs:
#                 if categ not in self.selected_attrs:
#                     continue
#                 for img_path in glob.glob(self.path + f"/{subfolder}/{categ}/*.jpg"):
#                     label = [(attr == categ) for attr in self.selected_attrs]
#                     dataset.append([img_path, label])

#         print(f'Finished preprocessing RAFD-format dataset at {self.path}: {len(self.train_dataset)} train images, {len(self.test_dataset)} test images')

#     def dataset_format(self):
#         return self.format

#     def label_size(self):
#         return len(self.selected_attrs)

#     def __len__(self):
#         dataset = self.train_dataset if self.mode == "train" else self.test_dataset
#         return len(dataset)

#     def __getitem__(self, index):
#         dataset = self.train_dataset if self.mode == "train" else self.test_dataset
#         filename, label = dataset[index]
#         image = Image.open(filename)
#         return self.transform(image), torch.FloatTensor(label)


# class VisualizationDataset(Dataset):
#     def __init__(self, path):
#         self.img_paths = []
#         for img_path in glob.glob(f'{path}/*.jpg'):
#             self.img_paths.append(img_path)

#         self.transform = get_default_transforms("test")

#     def __len__(self):
#         return len(self.img_paths)

#     def __getitem__(self, index):
#         filename = self.img_paths[index]
#         image = Image.open(filename)
#         return self.transform(image)


# def get_default_transforms(mode):
#     transform = []
#     if mode == "train":
#         transform.append(T.RandomHorizontalFlip())
#     transform.append(T.CenterCrop(178))
#     transform.append(T.Resize(128))
#     transform.append(T.ToTensor())
#     transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
#     transform = T.Compose(transform)
#     return transform


# def get_loader(dataset, batch_size, mode='train', num_workers=1):
#     """Build and return a data loader."""
#     data_loader = DataLoader(dataset=dataset,
#                             batch_size=batch_size,
#                             shuffle=(mode=='train'),
#                             num_workers=num_workers)
#     return data_loader


In [ ]:
from generic_dataset import GenericDataset, VisualizationDataset, get_loader
from generic_dataset import CELEBA_FORMAT_DATASET, RAFD_FORMAT_DATASET
from custom_solver import CustomSolver

In [ ]:
from pathlib import Path

project_root_dir = Path("/kaggle/input/celeba-dataset")
celeba_dir = project_root_dir
visualization_dir = Path("/kaggle/working/vis/")

In [ ]:
# Celeba dataset
celeba_selected_attrs = ["Male", "Young", "Black_Hair", "Blond_Hair", "Brown_Hair"]
celeba = GenericDataset(str(celeba_dir), CELEBA_FORMAT_DATASET, celeba_selected_attrs, "train")

Finished preprocessing CelebA-format dataset at /kaggle/input/celeba-dataset: 198548 train images, 4051 test images


In [ ]:
import shutil
import os
os.makedirs("/kaggle/working/vis")
cnt = 0
for i in range(1, 200000, 1000):
    cnt += 1
    img_name = str(i).zfill(6)
    src = f'/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/{img_name}.jpg'
    dest = f'/kaggle/working/vis/{cnt+1}.jpg'
    shutil.copyfile(src, dest)

In [ ]:
# Visualization dataset
vis_dataset = VisualizationDataset(visualization_dir)
print(len(vis_dataset))

200


In [ ]:
os.makedirs("/kaggle/working/exps/")

In [ ]:
settings = {
    "num_iters": 20000,
    "num_iters_decay": 0,
    "batch_size": 64,
    "lr_update_step": 2,
    "n_critic": 5,
    "log_step": 500,
    "vis_step": 5000,
    "model_save_step": 5000,
}

In [ ]:
# Train 1000 iters only on Celeba
solver = CustomSolver([celeba], vis_dataset, Path("/kaggle/working/exps/"), **settings)

sum_label_size: 5, mask_vector_size: 1
Zero padding left: [0]
Zero padding right: [0]
Workdir set at /kaggle/working/exps
Generator(
  (main): Sequential(
    (0): Conv2d(9, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ResidualBlock(
      (main): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_

In [ ]:
# 14 apr 2024 20:19
solver.train_multi()

Start training...
Elapsed [0:05:17], Iteration [500/20000], Dataset [1], D/loss_real: -11.0378, D/loss_fake: 5.7954, D/loss_cls: 2.5621, D/loss_gp: 0.0562, G/loss_fake: -5.2391, G/loss_rec: 0.2793, G/loss_cls: 3.5305
Elapsed [0:10:34], Iteration [1000/20000], Dataset [1], D/loss_real: -9.4586, D/loss_fake: 5.6576, D/loss_cls: 1.8490, D/loss_gp: 0.0258, G/loss_fake: -4.8365, G/loss_rec: 0.2612, G/loss_cls: 2.6164
Elapsed [0:15:51], Iteration [1500/20000], Dataset [1], D/loss_real: -7.0314, D/loss_fake: 3.2213, D/loss_cls: 1.4697, D/loss_gp: 0.0398, G/loss_fake: -2.8455, G/loss_rec: 0.2561, G/loss_cls: 1.5486
Elapsed [0:21:08], Iteration [2000/20000], Dataset [1], D/loss_real: -5.2789, D/loss_fake: 1.3641, D/loss_cls: 1.5574, D/loss_gp: 0.0424, G/loss_fake: -0.5763, G/loss_rec: 0.2051, G/loss_cls: 1.7158
Elapsed [0:26:25], Iteration [2500/20000], Dataset [1], D/loss_real: -5.9204, D/loss_fake: 3.1334, D/loss_cls: 1.3650, D/loss_gp: 0.0136, G/loss_fake: -3.7567, G/loss_rec: 0.1980, G/loss

In [ ]:
!zip -r file.zip /kaggle/working/exps/checkpoints/20000-G.ckpt


  adding: kaggle/working/exps/checkpoints/1000-G.ckpt (deflated 8%)


In [ ]:
!zip -r file.zip /kaggle/working/exps/checkpoints/20000-D.ckpt